In [ ]:
#from google.colab import drive
#drive.mount('/content/drive') 

In [ ]:
#!pip install -q condacolab 
#import condacolab
#condacolab.install()

In [ ]:
#!mamba install -q openmm 
#!mamba install -q mdtraj  
#!mamba install -q mdshare 
#!mamba install -q nglview
#!mamba install -q pyemma 
#!mamba install -q msmtools 

The tutorial aims to build a coarse-grained model of the B1 immunoglobulin-binding domain of streptococcal protein G (PDB code 1PGB, further referred to as protein G) that can capture folding-unfolding events.

**Task 1**

In this task, we will build a structure-based model of protein G and perform a short simulation. We will represent each residue as a single atom with unit mass, with each pair of consecutive atoms connected by a bond. Interactions between atoms are defined as followed:

$$
V = V_{bond} + V_{angle} + V_{dihedral} + V_{nonbonded}
$$

$$
V_{bond} = \sum_{bond} \frac{Kr}{2} (r - r_0)^2
$$

$$
V_{angle} = \sum{angle} \frac{K_{\theta}}{2} (\theta - \theta_0)^2
$$

$$
V_{dihedral} = \sum_{dihedral} [(1 - cos(\phi - \phi_0)) + 0.5 (1 - cos(3*(\phi - \phi_0))]
$$

$$
V_{nonbonded} = \sum_{native} 5 \left(\frac{r_0}{2}  \right)^{12} - 6\left(\frac{r_0}{2}  \right)^{10} + \sum_{non-native} \left(\frac{\sigma}{r}  \right)^{12}
$$   
In these equations, $r_0, \theta_0 , \Psi_0$, represent distances, angles and dihedral angles found in native structure. Values for $K_{\theta}, \sigma, K_r$ you can find in seed_notebook. We will build this model using tools provided by the OpenMM package.  

**1.1)** You are given a file '1PGB.pdb'. It is a pdb file that contains native structure of protein G. Using mdtraj, create a pdb file that contains only $C\alpha$ atoms and their respective positions. This file represents a topologynof our model. **Hint:** mdtraj.trajectory.atom_slice method may be useful(1 pt).

In [18]:
from sys import stdout
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
import numpy as np
import mdtraj as md
import pandas
import matplotlib.pyplot as plt
import nglview
import pdbfixer

In [19]:
#fixer = pdbfixer.PDBFixer(pdbid='1PGB')

In [20]:
#fixer.findMissingResidues()
#print(fixer.missingResidues)

#fixer.findNonstandardResidues()
#print(fixer.nonstandardResidues)

#fixer.findMissingAtoms()
#print(fixer.missingAtoms)

In [21]:
main_pdb = md.load_pdb('1PGB.pdb')

In [22]:
atoms_to_keep = [atom.index for atom in main_pdb.topology.atoms if atom.name == 'CA']

In [23]:
new_pdb = main_pdb.atom_slice(atoms_to_keep)

In [24]:
new_pdb.save('CA_only.pdb')

In [ ]:
#atoms, bonds = coord.topology.to_dataframe()
#atoms

In [ ]:
#atoms_to_keep = [a.index for a in coord.topology.atoms if a.name == 'CA']
#new_coord = coord.restrict_atoms(atoms_to_keep)
#coord.save('CA-only.pdb')

In [ ]:
#atoms, bonds = new_coord.topology.to_dataframe()
#atoms

**1.2)** Create a simtk.openmm.app.topology.Topology object. Add a chain, all the residues, one CA atom per residue, and bonds between atoms from consecutive residues (1 pt).

In [25]:
pdb = PDBFile('CA_only.pdb')
#pdb = md.load('CA_only.pdb')
#main_pdb = PDBFile('1PGB.pdb')

In [26]:
forcefield = ForceField('amber99sbildn.xml', 'tip3p.xml')

In [13]:
top = pdb.getTopology()
bonds = list(top.bonds())
for b in bonds:
    names = [b.atom1.name, b.atom2.name]
    if 'CA' in names:
        print(names, b.atom1.index, b.atom2.index)
    else:
        print('no Ca')

In [47]:
top = topology.Topology

In [66]:
chain=top.addChain(id='A')

TypeError: addChain() missing 1 required positional argument: 'self'

In [73]:
residue = top.addResidue(MET, 0)

NameError: name 'MET' is not defined

In [16]:
top.addResidue()

TypeError: addResidue() missing 3 required positional arguments: 'self', 'name', and 'chain'

In [17]:
system = forcefield.createSystem(pdb.topology)
#integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)

ValueError: No template found for residue 1 (MET).  This might mean your input topology is missing some atoms or bonds, or possibly that you are using the wrong force field.